<a href="https://colab.research.google.com/github/taruj/Advanced-Regression-Assignment/blob/main/Advanced_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem Statement
`Surprise Housing `a US Based Housing company plans to enter the Australian market. The company is looking at prospective properties to buy to enter the market.

Before investing the company would like to `predict the value` of the prospective properties and then decide to invest (or not). 

The company uses data analytics to purchase houses at a price below their actual values and flip them at a higher price. The company would also like to know `which factors (features of the houses) that are significant` in predicting the price of a house, and what is the impact of those variables in deciding the price of the house.

### Technical Goals
Build a regression model using regularisation in order to predict the actual value of the prospective properties and decide whether to invest.
Determine the optimal value of lambda for ridge and lasso regression.
This model will then be used by the management to understand how exactly the prices vary with the variables

### Approach for the Solution
- Exploratory Data Analysis 
- Data cleaning
- Data preparation 
  - Encoding, standardization and normalization
- Model building
- Conclusions

In [4]:
# Importing the required libraries
import numpy as np
import pandas as pd

# Vizulization
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

# Suppress Warning
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor


## Loading the Dataset

from google.colab import drive
drive.mount('/content/drive')

data =  pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Adv-Reg-Assign.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
 ## Get basic information on the data set
data.shape

(1460, 81)